In [31]:
import pymysql
import pandas as pd

# 공통
%matplotlib inline
import numpy as np
import statsmodels.api as sm

# 예측 1
import matplotlib.pyplot as plt
import datetime
from dateutil.relativedelta import relativedelta
import seaborn as sns
from statsmodels.tsa.stattools import acf
from statsmodels.tsa.stattools import pacf
from statsmodels.tsa.seasonal import seasonal_decompose

# 예측 2
from scipy.stats import norm
from tqdm import tqdm
from sklearn.metrics import mean_squared_error

In [43]:
con = pymysql.connect(host = '127.0.0.1', user = 'root', password = 'kyjin0413!',
                       db = 'lpointsample', charset = 'utf8')
cur = con.cursor(pymysql.cursors.DictCursor)
cur.execute(
    "select de_dt as 날짜, de_hr as 시간대, sum(buy_am) as 금액 ,count(*) as 이용횟수 from 엘페이이용_view group by de_dt, de_hr order by de_dt, de_hr")
result=cur.fetchall()
con.commit()
con.close()
df= pd.DataFrame(result)
# print(df.head(10))

df.columns = ['date', 'time', 'price', 'count_days']
# df = df.drop(['date'], axis = 1)
df.head()

,date,time,price,count_days
0,2021-01-01,0,2683380,72
1,2021-01-01,1,499360,11
2,2021-01-01,2,64176,10
3,2021-01-01,3,885833,11
4,2021-01-01,5,37630,1


In [53]:
df.to_csv('시계열_일별시간대.csv', header=['date', 'time', 'price', 'count_days'], index=False)

In [50]:
# # 인덱스 지정(?)

# # strptime은 날짜와 시간 형식의 문자열을 datetime으로 변환해 줌
# # strftime은 날짜와 시간(datetime)을 문자열로 출력
# start = datetime.datetime.strptime("2021-01-01", "%Y-%m-%d")

# # 기준 날짜에 대한 연산이 필요할 때 dateutil.relativedelta모듈의 relativedelta함수 이용
# df_list = [start + relativedelta(days = x) for x in range(0, len(df)) ]

# df['index'] = df_list

# # set_index는 데이터프레임을 먼저 불러온 후, 특정 열을 인덱스로 설정할 때 사용
# df.set_index( ['index'], inplace=True) 

# # index 출력 X
# df.index.name=None
# df.head()

,date,time,price,count_days
2021-01-01,2021-01-01,0,2683380,72
2021-01-02,2021-01-01,1,499360,11
2021-01-03,2021-01-01,2,64176,10
2021-01-04,2021-01-01,3,885833,11
2021-01-05,2021-01-01,5,37630,1


In [22]:
df.dtypes

week     object
price     int64
dtype: object

In [23]:
# price가 object임
df = df.astype({'price':'int64'})

In [24]:
df.dtypes

week     object
price     int64
dtype: object

In [25]:
# coef값은 0에서 가장 떨어졌을때 가장 이상적
# p 값은 0과 가장 가까울때가 가장 좋습니다

mod = sm.tsa.statespace.SARIMAX(
    df['price'],
    order = (1, 1, 1),
    seasonal_order = (0, 1, 1, 7),
    # AR 매개 변수를 변환하여 모델의 자동 회귀 구성 요소에서 정상 성을 강제 적용할지 여부입니다. 기본값은 True입니다.
    # AR항이 stationary를 만족하게끔 강제하는 것
    enforce_stationarity = False,
    # 모델의 이동 평균 구성 요소에서 반전 성을 강제하기 위해 MA 매개 변수를 변환할지 여부입니다. 기본값은 True입니다.
    # MA항이 starionary를 만족하게끔 강제하는 것
    enforce_invertibility = False
)

results = mod.fit()

# print(results.summary())
print( results.summary().tables[1] )

                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.7156      0.258      2.777      0.005       0.211       1.221
ma.L1         -0.4263      0.317     -1.347      0.178      -1.047       0.194
ma.S.L7       -0.8863      0.076    -11.711      0.000      -1.035      -0.738
sigma2       2.81e+15        nan        nan        nan         nan         nan


In [26]:
# Get forecast some steps ahead in future 미래 예측
pred_uc = results.get_forecast( steps = 31 )

In [27]:
forecast_mean = pred_uc.predicted_mean
forecast_mean

168    8.293867e+07
169    7.446518e+07
170    8.055884e+07
171    7.048978e+07
172    6.625277e+07
173    6.242284e+07
174    6.640621e+07
175    6.469324e+07
176    6.143684e+07
177    7.126397e+07
178    6.386666e+07
179    6.154161e+07
180    5.907992e+07
181    6.404243e+07
182    6.303016e+07
183    6.027520e+07
184    7.046116e+07
185    6.332064e+07
186    6.117936e+07
187    5.884918e+07
188    6.390579e+07
189    6.296086e+07
190    6.025410e+07
191    7.047455e+07
192    6.335871e+07
193    6.123509e+07
194    5.891755e+07
195    6.398321e+07
196    6.304475e+07
197    6.034262e+07
198    7.056639e+07
Name: predicted_mean, dtype: float64

In [28]:
forecast_mean.to_csv('lpoint_week.csv', header=['forecast'], index=False)